In [1]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# def supertrend(data, periods=10, multiplier=3.0, change_atr=True, show_signals=True, highlighting=True):
#     atr_col = 'ATR'
#     up_col = 'UpTrend'
#     dn_col = 'DownTrend'
#     trend_col = 'Trend'
#     buy_col = 'BuySignal'
#     sell_col = 'SellSignal'

#     data['ATR2'] = data['Close'].rolling(periods).mean()
#     data[atr_col] = data['ATR'] if change_atr else data['ATR2']

#     data[up_col] = data['Close'] - (multiplier * data[atr_col])
#     mask = data['Close'].shift(1) > data[up_col].shift(1)
#     data.loc[mask, up_col].iloc[1:] = np.maximum(data[up_col].shift(1).loc[mask], data[up_col].iloc[1:])

#     data[dn_col] = data['Close'] + (multiplier * data[atr_col])
#     data[dn_col].iloc[1:] = np.where(data['Close'].shift(1) < data[dn_col].shift(1), 
#                                      np.minimum(data[dn_col].iloc[1:], data[dn_col].shift(1).iloc[1:]), 
#                                      data[dn_col].iloc[1:])

#     data[trend_col] = 1
#     data[trend_col].iloc[1:] = np.where((data[trend_col].shift(1) == -1) & (data['Close'] > data[dn_col].shift(1)),
#                                         1,
#                                         np.where((data[trend_col].shift(1) == 1) & (data['Close'] < data[up_col].shift(1)),
#                                                  -1,
#                                                  data[trend_col].shift(1)))

#     buy_signal = (data[trend_col] == 1) & (data[trend_col].shift(1) == -1)
#     data[buy_col] = np.where(buy_signal, data[up_col], np.nan)

#     sell_signal = (data[trend_col] == -1) & (data[trend_col].shift(1) == 1)
#     data[sell_col] = np.where(sell_signal, data[dn_col], np.nan)

#     # Plotting
#     plt.figure(figsize=(10, 6))
#     plt.plot(data['Close'], label='Close Price')
#     plt.scatter(data.index[buy_signal], data[buy_col], marker='^', color='g', label='Buy Signal')
#     plt.scatter(data.index[sell_signal], data[sell_col], marker='v', color='r', label='Sell Signal')

#     for i in range(1, len(data)):
#         if data[trend_col].iloc[i] == 1:
#             plt.fill_between(data.index[i-1:i+1], data[up_col].iloc[i-1:i+1], data[dn_col].iloc[i-1:i+1], color='green', alpha=0.3)
#         elif data[trend_col].iloc[i] == -1:
#             plt.fill_between(data.index[i-1:i+1], data[up_col].iloc[i-1:i+1], data[dn_col].iloc[i-1:i+1], color='red', alpha=0.3)

#     plt.title('Supertrend Indicator')
#     plt.xlabel('Date')
#     plt.ylabel('Price')
#     plt.legend()
#     plt.show()


In [2]:
# # Assuming you have OHLC data in a DataFrame named 'df'
# # Replace this with your actual data
# df = pd.DataFrame({
#     'Open': [100, 110, 120, 130, 140],
#     'High': [105, 115, 125, 135, 145],
#     'Low': [95, 105, 115, 125, 135],
#     'Close': [102, 112, 122, 132, 142],
#     'ATR': [2, 2.1, 2.2, 2.3, 2.4]
# }, index=pd.date_range(start='2022-01-01', periods=5))

# # Call the supertrend function with your DataFrame
# supertrend(df)

In [4]:
# Importing required libraries
import talib
import numpy as np
import pandas as pd
import plotly.graph_objs as go

# Defining function for Supertrend
def supertrend(df, period=10, multiplier=3):
    """
    Function to compute Supertrend indicator
    """
    # Calculating ATR
    df['ATR'] = talib.ATR(df['High'].values, df['Low'].values, df['Close'].values, timeperiod=period)

    # Calculating Upper and Lower Band
    df['Upper Band'] = df['High'] - (multiplier * df['ATR'])
    df['Lower Band'] = df['Low'] + (multiplier * df['ATR'])

    # Initialising columns for data
    df['Trend'] = 0
    df['Trend'] = np.where(df['Close'] > df['Lower Band'], 1, 0)
    df['Trend'] = np.where(df['Close'] < df['Upper Band'], -1, df['Trend'])

    first_index = df['Trend'].first_valid_index()

    # Modifying Trend column based on Buy and Sell signals
    for i in range(first_index + 1, first_index + len(df)):
      if df.loc[i, 'Trend'] == 0:
          df.loc[i, 'Trend'] = df.loc[i-1, 'Trend']

      if df.loc[i, 'Close'] > df.loc[i-1, 'Lower Band']:
          df.loc[i, 'Trend'] = 1

      if df.loc[i, 'Close'] < df.loc[i-1, 'Upper Band']:
          df.loc[i, 'Trend'] = -1

    # Returning Supertrend values
    return df['Trend'], df['Upper Band'], df['Lower Band']

# Generating sample data
df = pd.read_csv('eurusd200323.csv').tail(1000)

# Calculating Supertrend values
trend, ub, lb = supertrend(df)

# Creating traces for candlestick chart and buy/sell signals
candlestick = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Candlestick')
buy_signal = go.Scatter(x=df.index[trend == 1], y=lb[trend == 1], mode='markers', name='Buy', marker=dict(color='green', size=10))
sell_signal = go.Scatter(x=df.index[trend == -1], y=ub[trend == -1], mode='markers', name='Sell', marker=dict(color='red', size=10))

# Creating layout for the plot
layout = go.Layout(title='Supertrend Indicator', xaxis=dict(title='Candlestick'), yaxis=dict(title='Price'))

# Creating figure object
fig = go.Figure(data=[candlestick, buy_signal, sell_signal], layout=layout)

# Displaying the plot
fig.show()